In [1]:
using LinearAlgebra, Plots
import ForwardDiff as FD
using Test
import Convex as cvx 
import ECOS
using Random

using JLD2


In [2]:
Ac = [0 0 1 0;
      0 0 0 1;
      0 0 0 0;
      0 0 0 0.]
Bc = [0 0;
      0 0;
      1 0;
      0 1]
nx, nu = size(Bc)
dt = 0.1
big_A = exp(Array([ Ac Bc; zeros(nu, nx) zeros(nu, nu) ]*dt))

6×6 Matrix{Float64}:
 1.0  0.0  0.1  0.0  0.005  0.0
 0.0  1.0  0.0  0.1  0.0    0.005
 0.0  0.0  1.0  0.0  0.1    0.0
 0.0  0.0  0.0  1.0  0.0    0.1
 0.0  0.0  0.0  0.0  1.0    0.0
 0.0  0.0  0.0  0.0  0.0    1.0

In [3]:
big_A[1:nx,1:nx], big_A[1:nx,nx+1:end]

([1.0 0.0 0.1 0.0; 0.0 1.0 0.0 0.1; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0], [0.005000000000000001 0.0; 0.0 0.005000000000000001; 0.1 0.0; 0.0 0.1])

In [4]:
A = big_A[1:nx,1:nx]
B = big_A[1:nx,nx+1:end]
x = [1.,2.,3.,4.]
u = [-1.,-3.]

A*x + B*u

4-element Vector{Float64}:
 1.2950000000000002
 2.385
 2.9
 3.7

In [5]:
a = [1 1; 0 3]

2×2 Matrix{Int64}:
 1  1
 0  3

In [6]:
exp(a)

2×2 Matrix{Float64}:
 2.71828   8.68363
 0.0      20.0855

In [7]:
N = 50

P = [zeros(nx,nx) for i = 1:N]
K = [zeros(nu,nx) for i = 1:N-1]

49-element Vector{Matrix{Float64}}:
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 ⋮
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]

In [8]:
P[1]

4×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [9]:
K[1]

2×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [10]:
Diagonal([1, 1, 0.1])

3×3 Diagonal{Float64, Vector{Float64}}:
 1.0   ⋅    ⋅ 
  ⋅   1.0   ⋅ 
  ⋅    ⋅   0.1

In [11]:
a = [1,2,3,4]
zeros(size(a))

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [12]:
# optimized trajectory we are going to try and track
DATA = load(joinpath(@__DIR__,"swingup.jld2"))
Xbar = DATA["X"]
Ubar = DATA["U"]

80-element Vector{Vector{Float64}}:
 [9.614479393105134]
 [8.631907099555065]
 [7.163824036935998]
 [5.3194237041436585]
 [3.312840338330581]
 [1.3757541505590203]
 [-0.37110577872350353]
 [-1.9703092314911446]
 [-3.590540062424266]
 [-5.4564102362369455]
 [-7.770700703834757]
 [-10.58137439976232]
 [-13.463578877058694]
 ⋮
 [-0.12945045705361616]
 [-0.1238189904856583]
 [-0.118094129571512]
 [-0.11231025852461198]
 [-0.10649123239483113]
 [-0.10064909458259305]
 [-0.09478238383443159]
 [-0.08887392614234986]
 [-0.08288798121751102]
 [-0.07676658134364998]
 [-0.07042486104086818]
 [-0.06374512741830562]

In [13]:
Ubar[1]

1-element Vector{Float64}:
 9.614479393105134

In [14]:
length(Ubar)

80